# Part 1 – Data collection

In [1]:
import numpy as np
import pandas as pd
import urllib.request
import networkx as nx
import matplotlib.pyplot as plt
import requests
import gseapy
import time
import csv
from bioservices import BioGRID,UniProt,PSICQUIC
from tqdm import tqdm_notebook

## 1.1) Explore information sources and compile the seed gene list:

In [2]:
# seed genes for HIV Infections(C0019693)

seed_genes = pd.read_csv('C0019693_disease_gda_summary_CURATED.tsv',sep='\t',usecols=['Gene'])['Gene'].values

In [3]:
len(seed_genes)

102

In [4]:
seed_genes

array(['IFNG', 'CD28', 'HLA-B', 'KIR3DL1', 'HLA-C', 'IL10', 'IL2', 'CCL2',
       'CXCL12', 'MBL2', 'IL2RA', 'CCL3', 'CCL3L1', 'CX3CR1', 'TRIM5',
       'SLC11A1', 'APOA1', 'SERPINA1', 'SIRT1', 'CD209', 'HP', 'CXCR1',
       'HSPB1', 'CDC42', 'CCR3', 'TLR3', 'ILF2', 'PPIB', 'MDM2', 'HSPD1',
       'PTCH1', 'CDKN1A', 'YY1', 'FCER2', 'IPO7', 'HSP90AA1', 'SFPQ',
       'RAB11B', 'SRSF1', 'CCT3', 'SRSF9', 'EIF3B', 'DNAJC7', 'SNRPD3',
       'SRF', 'TAF9', 'U2AF1', 'BAG3', 'GRPEL1', 'ARAP3', 'PSME4', 'CCT5',
       'RAB32', 'COPS6', 'SF3A3', 'HSPH1', 'PRPF8', 'CLIC1', 'EMG1',
       'SIGMAR1', 'NUTF2', 'PSME3', 'ALYREF', 'PSMD6', 'CCT7', 'CCL11',
       'LYN', 'EIF6', 'IL4R', 'HSP90AB1', 'HSPA9', 'HDGF', 'GTF3A',
       'EIF4EBP1', 'EIF4A1', 'GADD45A', 'BAX', 'ARHGDIB', 'ARHGDIA',
       'ARHGAP4', 'BIRC3', 'BIRC2', 'ALB', 'KPNB1', 'MAZ', 'RANBP1',
       'RAN', 'PSME2', 'PSME1', 'PSMD13', 'PSMD8', 'PSMD3', 'PSMC5',
       'PSMC3', 'PSMB10', 'PSMB4', 'PSMA6', 'PSMA5', 'PSMA4', 'ODC1',
     

In [5]:
def uniprot_AC(genes):
    """
    this function return the uniprot_AC code given a gene symbol
    """
    ids = []
    HGNC = pd.read_csv('hgnc_complete_set.txt',sep='\t',usecols=['symbol','uniprot_ids']) 
    u = UniProt(verbose=False)
    if type(genes) == str:
        genes = [genes]
    for gene in tqdm_notebook(genes):
        try:   
            if not HGNC[HGNC.symbol==gene]['uniprot_ids'].isna().values[0]:
                gene_id =  HGNC[HGNC.symbol==gene]['uniprot_ids'].values[0]
            else:         
                gene_id = u.search(gene, columns="id").split('\n')[1]
        except:
            gene_id = '-'
        ids.append(gene_id)
    return(ids)


def gene_details(gene):
    """
    this function return the genes information given a gene symbol
    """
    query = 'https://www.uniprot.org/uniprot/?query=gene_exact:{}+AND+reviewed:yes+AND+organism:%22Homo%20sapiens%20(Human)%20[9606]%22&columns=id,database(GeneID),comment(FUNCTION),protein%20names&format=tab'.format(gene)
    page = urllib.request.urlopen(query)
    page = page.read().decode()
    attrs = page.split('\t')
    
    uniprot_AC = attrs[3].split('\n')[1]
    
    s = attrs[-1]
    protein_name = s[:s.find("(")]
    
    entrez_gene_id = attrs[4].strip(';')
    
    s = attrs[5].split(': ')[1]
    description = s[:s.find(" {")]
    
    
    time.sleep(2)
    return(gene,uniprot_AC,protein_name,entrez_gene_id,description)
        

In [6]:
df = pd.DataFrame(columns=['seed Gene','Uniprot_AC','protein_name','entrez_gene_id','description'])
for i in tqdm_notebook(range(len(seed_genes))):
    try:
        df.loc[i] = gene_details(seed_genes[i])
    except:
        pass

#### (check if the symbols are updated and approved on the HGNC website; report any issue/lack of data/potential misinterpretation)

In [7]:
HGNC = pd.read_csv('hgnc_complete_set.txt',sep='\t',usecols=['symbol','name','status','uniprot_ids'])
HGNC.head()

,symbol,name,status,uniprot_ids
0,A1BG,alpha-1-B glycoprotein,Approved,P04217
1,A1BG-AS1,A1BG antisense RNA 1,Approved,NaN
2,A1CF,APOBEC1 complementation factor,Approved,Q9NQ94
3,A2M,alpha-2-macroglobulin,Approved,P01023
4,A2M-AS1,A2M antisense RNA 1,Approved,NaN


In [8]:
for gene in seed_genes:
    try:
        if not HGNC[HGNC.symbol==gene]['status'].values[0]=='Approved':
            print('Error')
            print(gene)
            print()
            df.drop(df[df['seed Gene'] == gene].index, inplace=True)
    except:
        print('Error')
        print(gene)
        print()
        df.drop(df[df['seed Gene'] == gene].index, inplace=True)
df = df.reset_index(drop=True)

In [9]:
df.head()

,seed Gene,Uniprot_AC,protein_name,entrez_gene_id,description
0,IFNG,P01579,Interferon gamma,3458,Produced by lymphocytes activated by specific ...
1,CD28,P10747,T-cell-specific surface glycoprotein CD28,940,"Involved in T-cell activation, the induction o..."
2,HLA-B,P01889,"HLA class I histocompatibility antigen, B alph...",3106,Antigen-presenting major histocompatibility co...
3,KIR3DL1,P43629,Killer cell immunoglobulin-like receptor 3DL1,3811,Receptor on natural killer (NK) cells for HLA ...
4,HLA-C,P10321,"HLA class I histocompatibility antigen, C alph...",3107,Antigen-presenting major histocompatibility co...


In [10]:
df.to_csv('Seed_Genes.csv',sep='\t',index=False)

## 1.2) Collect interaction data

### BioGrid_Database

In [37]:
total_node=[]
for gene in tqdm_notebook(seed_genes):
    try:
        b = BioGRID(query=gene,taxId="9606")  ##9606 for Humans
        total_node.append(b.biogrid.interactors)
        time.sleep(2)
    except:
        print('ERROR, gene {} not found!'.format(gene))


total_genes_Biogrid = []
seed_genes_in_DB_Biogrid = 0
for t in total_node:
    if len(t) > 0:
        seed_genes_in_DB_Biogrid += 1
        total_genes_Biogrid.extend(t)

ERROR, gene SLC11A1 not found!



In [38]:
# remove interactions featuring non-primary seed gene symbols, non valid symbols, or spurious interactions
# including interactions mentioned with Alias

total_genes_updated = []
non_seed_genes = []

for tpl in total_genes_Biogrid:
    if tpl[0] in seed_genes or tpl[1] in seed_genes:
        if "/" not in tpl[0] and "/" not in tpl[1]:
            total_genes_updated.append(tpl)
        
total_genes_Biogrid = total_genes_updated.copy()


In [39]:
all_genes_interacting = []
for x in total_genes_Biogrid:
    all_genes_interacting += [*x]

In [40]:
# seed genes skipped
len(seed_genes) - len(set(all_genes_interacting).intersection(set(seed_genes)))

25

In [41]:
seed_genes_skipped = list(set(seed_genes).difference(set(all_genes_interacting)))
#delete SLC11A1  gene
seed_genes_skipped.remove('SLC11A1')
print(len(seed_genes_skipped))
print(seed_genes_skipped) # discarded seed genes

24
['CDC42', 'HLA-B', 'CCT3', 'ILF2', 'IL10', 'CLIC1', 'IL4R', 'SIGMAR1', 'MBL2', 'ALB', 'SIRT1', 'SRSF1', 'EIF6', 'TAF9', 'HDGF', 'SERPINA1', 'HLA-C', 'PTCH1', 'KIR3DL1', 'HSPH1', 'GADD45A', 'GTF3A', 'IL2RA', 'RAN']


In [42]:
# using PSICQUIC package from bioservices 
# to find skipped seed genes - non seed genes interactions


queries = []
new_seed_interactions = []

psic = PSICQUIC(verbose=False)

for gene in seed_genes_skipped:
    queries.append((gene+" AND species:9606"))


for query in tqdm_notebook(queries):
    print(query)
    data = psic.query("biogrid", query)
    for item in data:
        new_seed_interactions.append([query.split(' ')[0], item[2]])
        
        #non_seed_interactions.append([query.split(' ')[0], item[3]])
        

CDC42 AND species:9606
HLA-B AND species:9606
CCT3 AND species:9606
ILF2 AND species:9606
IL10 AND species:9606
CLIC1 AND species:9606
IL4R AND species:9606
SIGMAR1 AND species:9606
MBL2 AND species:9606
ALB AND species:9606
SIRT1 AND species:9606
SRSF1 AND species:9606
EIF6 AND species:9606
TAF9 AND species:9606
HDGF AND species:9606
SERPINA1 AND species:9606
HLA-C AND species:9606
PTCH1 AND species:9606
KIR3DL1 AND species:9606
HSPH1 AND species:9606
GADD45A AND species:9606
GTF3A AND species:9606
IL2RA AND species:9606
RAN AND species:9606



In [43]:
# seeking synonyms
for x in new_seed_interactions:
    if len(x[1].split('gene name synonym')) > 1:
        print('gene synonym')
        
new_seed_interactions[0][1].split('locuslink:')

['biogrid:111095|entrez gene/', 'PAK1']

In [44]:
new_seed_interactions[0][1].split('locuslink:')

['biogrid:111095|entrez gene/', 'PAK1']

In [45]:
new_seed_interactions[1][1].split('locuslink:')

['biogrid:107433|entrez gene/', 'CDC42|entrez gene/', 'RP1-224A6.5']

In [46]:
updated_list = []

for item in new_seed_interactions:
    
    splitted = item[1].split('locuslink:')
    
    if  len(splitted) == 2:
        # only one interactor
        updated_list.append([item[0], splitted[1]])
        
    elif len(splitted) == 3:
        updated_list.append([item[0], splitted[1].split('|')[0]])
        updated_list.append([item[0], splitted[2]])     

print(len(updated_list))

6365


In [47]:
new_seed_interactions = updated_list.copy()

In [48]:
#old size of interactions
len(total_genes_Biogrid)

8746

In [49]:
#genes we add to the list
len(new_seed_interactions)

6365

In [50]:
#update all_possible interactions list of Biogrid
for k in new_seed_interactions:
    total_genes_Biogrid.append((k[0],k[1]))

In [51]:
#new size of all possible interactions
len(total_genes_Biogrid)

15111

In [52]:
# collect non-seed genes interacting with seed genes

for tpl in total_genes_Biogrid:
    if tpl[0] in seed_genes and tpl[1] not in seed_genes:
        non_seed_genes.append(tpl[1])
    elif tpl[0] not in seed_genes and tpl[1] in seed_genes:
        non_seed_genes.append(tpl[0])
        
non_seed_genes = list(set(non_seed_genes))

In [53]:
#number of non seed genes
len(non_seed_genes)

4992

In [54]:
# non seed interactions list
non_seed_interactions = []

k = 0
step = 50
k_max = len(non_seed_genes) - len(non_seed_genes)%step

# This function gets interactions among the non-seed genes
# from the index k1 to the index k2 of the non seed genes list
# using the BioGRID REST Service

def obtain_interactors(k1, k2):
    
    # set first result to get from the results list returned by BioGRID REST
    start = 0
    # set number of results to the maximum allowed by BioGrid REST
    nresults = 10000
    
    gn_lst = non_seed_genes[k1:k2].copy()
    gn_lst = "|".join(gn_lst)
    
    interactors = []
    
    # while 10000 or more results are returned
    # (so we can expect the existence of more results)
    while nresults >= 10000:
        
        # execute http request and parse result
        query="https://webservice.thebiogrid.org/interactions?searchNames=true&start=" + str(start) + "&geneList=" + gn_lst + "&includeInteractors=true&includeInteractorInteractions=false&taxId=9606&accesskey=87c0c25a7e5df5b0e8fdac37eb222ad2"

        page = urllib.request.urlopen(query)
        page = page.read().decode()
        lines = page.split('\n')
        
        # if more than 10000 new results are found
        # update starting point
        nresults = len(lines)
        if nresults >= 10000:
            start = start + 10000

        # get interactions
        for line in lines[0:len(lines) - 1]:
            items = line.split("\t")
            interactors.append((items[7], items[8]))

    # check and select only interactions between non seed genes
    checked_interactors = []
    
    for tpl in interactors:
        if tpl[0] in non_seed_genes and tpl[1] in non_seed_genes:
            checked_interactors.append(tpl)
            
    # return non seed interactions
    return checked_interactors

# while the list of non seed genes is not completely covered 
while k < k_max:
    
    # obtain interactions for group of non seed genes
    interactors = obtain_interactors(k, k+step)
    # extend non seed interactions
    non_seed_interactions.extend(interactors)
    # proceed further in the non seed genes list
    k = k + step
    time.sleep(2)

# cover the last chunk of non seed genes
if k_max < len(non_seed_genes):
    interactors = obtain_interactors(k, len(non_seed_genes))
    
# extend non seed interactions
non_seed_interactions.extend(interactors)

In [56]:
total_genes_Biogrid.extend(non_seed_interactions)
print(len(total_genes_Biogrid))

290754


In [57]:
#BioGrid interactions export
with open('BioGrid.txt', 'w', newline='') as f:
    for c in total_genes_Biogrid:
        f.write('{} {}\n'.format(c[0],c[1]))
    f.close()

In [58]:
df_Biogrid = pd.DataFrame(data=total_genes_Biogrid,columns=['symbol1','symbol2'])

In [59]:
G_Biogrid = nx.Graph()
G_Biogrid.add_nodes_from(seed_genes)
G_Biogrid.add_nodes_from(non_seed_genes)
G_Biogrid.add_edges_from(total_genes_Biogrid)

### IID Integrated Interactions Database

In [60]:
IID_Database = pd.read_csv('human_annotated_PPIs.txt',sep='\t',usecols=['uniprot1','uniprot2','symbol1','symbol2','dbs','evidence type'])
IID_Database.head()

,uniprot1,uniprot2,symbol1,symbol2,dbs,evidence type
0,Q9NUX5,Q9NVM4,POT1,PRMT7,biogrid;intact,exp
1,Q96JY6,Q9NPC6,PDLIM2,MYOZ2,iid-pred,pred
2,Q15414,Q32P51,-,HNRNPA1L2,iid-pred,pred
3,P62633,Q99729,CNBP,HNRNPAB,biogrid;intact,exp
4,P29353,P42685,SHC1,FRK,iid-pred,pred


In [61]:
IID_Database = IID_Database[IID_Database['evidence type'].str.contains("exp")] 
IID_Database = IID_Database[IID_Database['dbs'].str.contains("iid")] 

In [62]:
df_IID = pd.DataFrame(columns=IID_Database.columns)

seed_genes_in_DB_IID = 0
for gene in seed_genes:
    temp = IID_Database[(IID_Database['symbol1'] == gene) | (IID_Database['symbol2'] == gene)]
    if len(temp)>0:
        seed_genes_in_DB_IID += 1
        df_IID = df_IID.append(temp)
    else:
        uniprot_id = uniprot_AC(gene)[0]
        temp = IID_Database[(IID_Database['uniprot1'] == uniprot_id) | (IID_Database['uniprot2'] == uniprot_id)]
        if len(temp)>0:
            seed_genes_in_DB_IID += 1
            for i in range(temp.shape[0]):
                if temp['uniprot1'].iloc[i] == uniprot_id:
                    temp['symbol1'].iloc[i] = gene
                else:
                    temp['symbol2'].iloc[i] = gene
            df_IID = df_IID.append(temp)
        else:
            print('ERROR, gene {} not found!'.format(gene))
        

df_IID = df_IID.reset_index(drop=True)
df_IID.head()

/home/jason/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/jason/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



ERROR, gene SLC11A1 not found!


,uniprot1,uniprot2,symbol1,symbol2,dbs,evidence type
0,P01579,P15260,IFNG,IFNGR1,bci;biogrid;dip;hprd;iid-pred;innatedb;intact,exp;pred
1,P01579,P38484,IFNG,IFNGR2,bci;biogrid;hprd;iid-pred;innatedb,exp;pred
2,P01579,Q9BZS1,IFNG,FOXP3,iid-ortho;intact,exp;ortho
3,P10747,P15498,CD28,VAV1,biogrid;iid-pred,exp;pred
4,P10747,P27986,CD28,PIK3R1,biogrid;hprd;iid-ortho;iid-pred;innatedb;intac...,exp;ortho;pred


In [63]:
non_seed_genes = []

for idx in range(len(df_IID)):
    if df_IID["symbol1"][idx] not in seed_genes:
        non_seed_genes.append(df_IID["symbol1"][idx])
    if df_IID["symbol2"][idx] not in seed_genes:
        non_seed_genes.append(df_IID["symbol2"][idx])
        
non_seed_genes = list(set(non_seed_genes))

temp = IID_Database[IID_Database["symbol1"].isin(non_seed_genes) & IID_Database["symbol2"].isin(non_seed_genes)] 

df_IID = df_IID.append(temp)
df_IID = df_IID.reset_index(drop=True)
df_IID.head()

,uniprot1,uniprot2,symbol1,symbol2,dbs,evidence type
0,P01579,P15260,IFNG,IFNGR1,bci;biogrid;dip;hprd;iid-pred;innatedb;intact,exp;pred
1,P01579,P38484,IFNG,IFNGR2,bci;biogrid;hprd;iid-pred;innatedb,exp;pred
2,P01579,Q9BZS1,IFNG,FOXP3,iid-ortho;intact,exp;ortho
3,P10747,P15498,CD28,VAV1,biogrid;iid-pred,exp;pred
4,P10747,P27986,CD28,PIK3R1,biogrid;hprd;iid-ortho;iid-pred;innatedb;intac...,exp;ortho;pred


In [64]:
#IID interactios export
total_genes_IID = []
for k in df_IID[['symbol1','symbol2']].values:
    total_genes_IID.append((k[0],k[1]))
    
with open('IID.txt', 'w', newline='') as f:
    for c in total_genes_IID:
        f.write('{} {}\n'.format(c[0],c[1]))
    f.close()

In [65]:
G_IID = nx.Graph()
G_IID.add_nodes_from(seed_genes)
G_IID.add_nodes_from(non_seed_genes)
G_IID.add_edges_from(total_genes_IID)

color = []
for node in G_IID:
    if node in seed_genes:
        color.append('blue')
    else: color.append('pink')

## Summarize the main results

In [66]:
gn_lst = [tpl[0] for tpl in total_genes_Biogrid]
gn_lst.extend([tpl[1] for tpl in total_genes_Biogrid])
gn_lst_Biogrid = set(gn_lst)

In [67]:
gn_lst = [tpl[0] for tpl in total_genes_IID]
gn_lst.extend([tpl[1] for tpl in total_genes_IID])
gn_lst_IID = set(gn_lst)

In [68]:
d = {'Number of seed genes' : [seed_genes_in_DB_Biogrid,seed_genes_in_DB_IID], 
     'Total Number of interacting proteins (including seed genes)' : [len(gn_lst_Biogrid),len(gn_lst_IID)],
        'Total Number of interactions' : [G_Biogrid.number_of_edges(),G_IID.number_of_edges()]}
pd.DataFrame(d, index=['BioGrid','IID (Integrated Interactions Database)'])

,Number of seed genes,Total Number of interacting proteins (including seed genes),Total Number of interactions
BioGrid,101,5093,98902
IID (Integrated Interactions Database),101,1782,21812


In [69]:
# this is a protein_symbol to uniprot_entry dictionary

keys = set()

keys.update(list(G_Biogrid.nodes))
keys.update(list(G_IID.nodes))

values = uniprot_AC(keys)
gene_to_uniprot = dict(dict(zip(keys,values)))

In [70]:
Summarized_Data = pd.DataFrame(columns=['interactor A gene symbol','interactor B gene symbol'],data=list(G_Biogrid.edges()))
Summarized_Data['interactor A Uniprot AC'] = Summarized_Data['interactor A gene symbol'].apply(lambda x:gene_to_uniprot[x])
Summarized_Data['interactor B Uniprot AC'] = Summarized_Data['interactor B gene symbol'].apply(lambda x:gene_to_uniprot[x])
Summarized_Data['database source'] = 'Biogrid'
temp = pd.DataFrame(columns=['interactor A gene symbol','interactor B gene symbol'],data=list(G_IID.edges()))
temp['interactor A Uniprot AC'] = temp['interactor A gene symbol'].apply(lambda x:gene_to_uniprot[x])
temp['interactor B Uniprot AC'] = temp['interactor B gene symbol'].apply(lambda x:gene_to_uniprot[x])
temp['database source'] = 'IID'


Summarized_Data = pd.concat([Summarized_Data,temp],ignore_index=True)
Summarized_Data.head()

,interactor A gene symbol,interactor B gene symbol,interactor A Uniprot AC,interactor B Uniprot AC,database source
0,IFNG,RP3-503F13.3,P01579,-,Biogrid
1,IFNG,GOPC,P01579,Q9HD26,Biogrid
2,IFNG,IFNGR2,P01579,P38484,Biogrid
3,IFNG,STAT6,P01579,P42226,Biogrid
4,IFNG,IFNG,P01579,P01579,Biogrid


In [71]:
Summarized_Data.to_csv('SummarizedData.csv',sep='\t',index=False)

## 1.3) Arrange interaction data

In [72]:
# a) Seed Genes Interactome

seed_genes_interactome = Summarized_Data[(Summarized_Data['interactor A gene symbol'].isin(seed_genes)) & (Summarized_Data['interactor B gene symbol'].isin(seed_genes))]
seed_genes_interactome.reset_index(drop=True)
seed_genes_interactome.to_csv('SeedGeneInteractome.csv',sep='\t',index=False)
seed_genes_interactome.head()

,interactor A gene symbol,interactor B gene symbol,interactor A Uniprot AC,interactor B Uniprot AC,database source
4,IFNG,IFNG,P01579,P01579,Biogrid
15,HLA-B,HLA-B,P01889,P01889,Biogrid
32,HLA-B,ADRB2,P01889,P07550,Biogrid
48,HLA-B,HLA-C,P01889,P10321,Biogrid
49,KIR3DL1,KIR3DL1,P43629,P43629,Biogrid


In [73]:
# b) Union Interactome 

union_interactome = Summarized_Data[['interactor A gene symbol', 'interactor B gene symbol', 'interactor A Uniprot AC', 'interactor B Uniprot AC']]
union_interactome.reset_index(drop=True)
union_interactome.drop_duplicates(inplace=True)
union_interactome.to_csv('UnionInteractome.csv',sep='\t',index=False)
union_interactome.head()

/home/jason/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,interactor A gene symbol,interactor B gene symbol,interactor A Uniprot AC,interactor B Uniprot AC
0,IFNG,RP3-503F13.3,P01579,-
1,IFNG,GOPC,P01579,Q9HD26
2,IFNG,IFNGR2,P01579,P38484
3,IFNG,STAT6,P01579,P42226
4,IFNG,IFNG,P01579,P01579


In [74]:
total_genes_Biogrid_rev = []
for tpl in total_genes_Biogrid:
    total_genes_Biogrid_rev.append((tpl[1], tpl[0]))

In [75]:
total_genes1 = [item for sublist in total_genes_Biogrid for item in sublist]

total_genes1 = set(total_genes1).intersection(set(total_genes_IID))
total_genes1 = list(total_genes1)
total_genes2 = set(total_genes_Biogrid_rev).intersection(set(total_genes_IID))
total_genes2 = list(total_genes2)
total_genes1.extend(total_genes2)
total_genes = list(set(total_genes1))

In [76]:
# c) Intersection Interactome

intersection_interactome = pd.DataFrame(columns=['interactor A gene symbol','interactor B gene symbol'],data=total_genes)
intersection_interactome['interactor A Uniprot AC'] = intersection_interactome['interactor A gene symbol'].apply(lambda x:gene_to_uniprot[x])
intersection_interactome['interactor B Uniprot AC'] = intersection_interactome['interactor B gene symbol'].apply(lambda x:gene_to_uniprot[x])
intersection_interactome.to_csv('IntersectionInteractome.csv',sep='\t',index=False)
intersection_interactome.head()

,interactor A gene symbol,interactor B gene symbol,interactor A Uniprot AC,interactor B Uniprot AC
0,DHX15,ARRB2,O43143,P32121
1,CAD,HDAC6,P27708,Q9UBN7
2,RPL7,RPS13,P18124,P62277
3,MYC,HSP90AA1,P01106,P07900
4,TP53,GNL3,P04637,Q9BVP2


## 1.4) Enrichment analysis

In [77]:
%%capture

# a) seed_genes

gseapy.enrichr(gene_list=list(seed_genes), description='gene_ontology', gene_sets='GO_Biological_Process_2018', outdir='seed_genes',cutoff=0.05, format='png')
gseapy.enrichr(gene_list=list(seed_genes), description='gene_ontology', gene_sets='GO_Cellular_Component_2018', outdir='seed_genes',cutoff=0.05, format='png')
gseapy.enrichr(gene_list=list(seed_genes), description='gene_ontology', gene_sets='GO_Molecular_Function_2018', outdir='seed_genes',cutoff=0.05, format='png')
gseapy.enrichr(gene_list=list(seed_genes), description='pathway', gene_sets='KEGG_2019_Human', outdir='seed_genes',cutoff=0.05, format='png')




In [80]:
# b) union_interactome_genes



U_I = set()
for k in union_interactome.values:
    for i in range(2):
        U_I.add(k[i])

U_I = list(U_I)

#Careful!! Maybe it should be wise to add a small time-out 
#between the executions in order to avoid any connection error...

gseapy.enrichr(gene_list=U_I, description='gene_ontology', gene_sets='GO_Biological_Process_2018', outdir='union_interactome_genes',cutoff=0.05, format='png')
gseapy.enrichr(gene_list=U_I, description='gene_ontology', gene_sets='GO_Cellular_Component_2018', outdir='union_interactome_genes',cutoff=0.05, format='png')
gseapy.enrichr(gene_list=U_I, description='gene_ontology', gene_sets='GO_Molecular_Function_2018', outdir='union_interactome_genes',cutoff=0.05, format='png')
gseapy.enrichr(gene_list=U_I, description='pathway', gene_sets='KEGG_2019_Human', outdir='union_interactome_genes',cutoff=0.05, format='png')